In [1]:
#ran with llm_comparisons new
import pandas as pd
import os

In [2]:
# Load the dataset and filename
dataset_task_mappings = pd.read_csv("../../dataset_task_mappings.csv", encoding='unicode_escape')

In [3]:
datasets = [1, 2, 3, 4]
task  = 2
test_size = 0.2
df_out = pd.DataFrame()
for dataset in datasets:
    print(f"Processing dataset {dataset}")
    #check which mappings to import
    dataset_idx = dataset_task_mappings.index[
        (dataset_task_mappings["dataset_number"] == dataset) & (dataset_task_mappings["task_number"] == task)]
    label_column = dataset_task_mappings.loc[dataset_idx, "label_column"].values[0]
    #CHECK THIS LATER - for some reason need to add[0] because it is recognizing an array, but didn't need to do before - some package versioning issue?
    df = pd.read_csv(os.path.join("../../data", dataset_task_mappings.loc[dataset_idx, "dataset_filename"].values[0]))
    print(df.columns)
    df = df.dropna(subset=[label_column])
    
    #print the distirbution of values
    print(df[label_column].value_counts())

    if dataset == 3:
        df['text'] = df['title_h1'] + ' ' + df['text_200']

    if dataset == 4:
        df['text'] = df['title'] + ' ' + df['text_250']
        df['problem_solution_ra'] = df[label_column]

    #remove duplicates - before and after merging
    n_rows_before_deduplication = len(df)
    df = df.drop_duplicates(subset=['text', 'problem_solution_ra'])
    print(f"Dataset has {len(df)} rows. {n_rows_before_deduplication - len(df)} duplicates were removed")

    #make column of dataset for validation
    df['dataset'] = dataset

    #append the dataframe
    df_out = pd.concat([df_out, df[['dataset', 'text', 'problem_solution_ra']]])

Processing dataset 1
Index(['status_id', 'Date', 'text', 'relevant_paula', 'relevant_fabio',
       'problem_frame_paula', 'problem_frame_fabio', 'solution_frame_paula',
       'solution_frame_fabio', 'relevance_agreement_check',
       'problem_solution_agreement_check', 'problem_solution_ra'],
      dtype='object')
problem_solution_ra
Neither     1462
Solution     339
Problem      200
Both         141
Name: count, dtype: int64
Dataset has 1754 rows. 388 duplicates were removed
Processing dataset 2
Index(['user_id', 'screen_name', 'status_id', 'created_at', 'text',
       'relevant_fabio', 'relevant_paula', 'problem_frame_fabio',
       'problem_frame_paula', 'solution_frame_fabio', 'solution_frame_paula',
       'relevance_agreement_check', 'problem_solution_agreement_check',
       'problem_solution_ra'],
      dtype='object')
problem_solution_ra
Neither     676
Solution     85
Problem      64
Name: count, dtype: int64
Dataset has 819 rows. 6 duplicates were removed
Processing datas

In [4]:
df_out['problem_solution_ra'].value_counts()

problem_solution_ra
Neither     3306
Solution     757
Problem      492
Both          80
Name: count, dtype: int64

In [5]:
duplicate_check = df_out.groupby(['text']).count().reset_index()

In [6]:
duplicate_check.loc[duplicate_check['dataset'] > 1, 'text'].values

array(["Congressman demands Facebook's #Zuckerberg explain why Donald #Trump Jr's TWITTER ACCOUNT was suspended\r\n\r\nMORE: https://t.co/HlJqGEB2Sk https://t.co/1MPtbY75Pd",
       "Hindustani Bhau's account was suspended. So Amaal Mallik and Bhai fans are working overtime to maintain the balance of gaalis in the universe.",
       'Someone is trying to ban my Instagram account because they stole my art and did not credit me for my art , What the actual fck, they have been harassing me for 2 weeks https://t.co/Jpnk06IU0E',
       'Twitter vaccine myths crackdown overdue When I was little, I was fond of a book called Girls Can Be Anything. It is about two friends. Adam insists only boys can be pilots and doctors, and Marina sets out to prove him wrong. When Adam\'s preposterous boast every U.S. president has been a man turns out to be true, Marina resolves to become the first female president.\r\n\r\n"This is the book that made me a feminist," begins a review on Goodreads.com, which ca

In [7]:
df_out[df_out['text'].isin(duplicate_check.loc[duplicate_check['dataset'] > 1, 'text'].values)]

,dataset,text,problem_solution_ra
235,1,Hindustani Bhau's account was suspended. So Am...,Neither
242,1,Someone is trying to ban my Instagram account ...,Problem
940,1,Hindustani Bhau's account was suspended. So Am...,Problem
1217,1,Congressman demands Facebook's #Zuckerberg exp...,Neither
1583,1,Someone is trying to ban my Instagram account ...,Neither
2324,1,Congressman demands Facebook's #Zuckerberg exp...,Problem
119,3,Twitter vaccine myths crackdown overdue When I...,Solution
1224,3,Twitter vaccine myths crackdown overdue When I...,Neither


In [8]:
df_out = df_out[~df_out['text'].isin(duplicate_check.loc[duplicate_check['dataset'] > 1, 'text'].values)]

In [9]:
#split into train and test
df_out.groupby('dataset').count()

,text,problem_solution_ra
dataset,,
1,1748,1748
2,819,819
3,1518,1521
4,536,539


In [10]:
df_out.groupby('problem_solution_ra').count()

,dataset,text
problem_solution_ra,,
Both,80,79
Neither,3302,3300
Problem,489,488
Solution,756,754


In [11]:
df_out = df_out.reset_index(drop = True)

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

# Create a combined stratification label
df_out['stratify_label'] = df_out['dataset'].astype(str) + '_' + df_out['problem_solution_ra']

# Initialize StratifiedShuffleSplit
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Perform split
for train_index, test_index in stratified_split.split(df_out, df_out['stratify_label']):
    train_set = df_out.loc[train_index]
    test_set = df_out.loc[test_index]

# Drop the temporary stratify_label column
test_set = test_set.drop(columns=['stratify_label'])

In [13]:
train_set.groupby('dataset').count()

,text,problem_solution_ra,stratify_label
dataset,,,
1,1399,1399,1399
2,655,655,655
3,1213,1216,1216
4,428,431,431


In [14]:
train_set.groupby('problem_solution_ra').count()

,dataset,text,stratify_label
problem_solution_ra,,,
Both,64,63,64
Neither,2641,2639,2641
Problem,391,390,391
Solution,605,603,605


In [15]:
test_set.groupby(['dataset', 'problem_solution_ra']).count()

text
dataset problem_solution_ra      
1       Both                    3
        Neither               260
        Problem                35
        Solution               51
2       Neither               135
        Problem                13
        Solution               16
3       Both                    9
        Neither               224
        Problem                22
        Solution               50
4       Both                    4
        Neither                42
        Problem                28
        Solution               34

In [16]:
train_set = train_set.reset_index(drop = True)

In [17]:
#  StratifiedShuffleSplit by both removes dataset completely because it's not a large enough group
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)

# Perform split
for train_index, test_index in stratified_split.split(train_set, train_set['stratify_label']):
    train_set_50 = train_set.loc[train_index]

train_set_50 = train_set_50.drop(columns=['stratify_label'])
train_set = train_set.drop(columns=['stratify_label'])

In [18]:
train_set_50.groupby('dataset').count()

,text,problem_solution_ra
dataset,,
1,700,700
2,327,327
3,606,608
4,214,215


In [19]:
train_set_50.groupby('problem_solution_ra').count()

,dataset,text
problem_solution_ra,,
Both,33,33
Neither,1320,1319
Problem,195,195
Solution,302,300


In [20]:
len(train_set)

3701

In [21]:
#export files
train_set_50.to_csv("../../data/ds_5__t_2_train_set_1850.csv")
train_set.to_csv("../../data/ds_5__t_2_train_set_3701.csv")
test_set.to_csv("../../data/ds5__t_2_eval_set.csv")